In [ ]:
# !pip install pyspark
# !pip install -q findspark
# !pip install pyspark-stubs
# !pip install pyfpgrowth
# !pip install modin[dask]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 46.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=7fce6642ad8b87435c913e64f3cb8f41056ee74f50d7e13f74bc915099aa7180
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 5.4 MB/s 
     |████████████████████████████████| 209.1 MB 53 kB/s 
     |████████████████████████████████| 198 kB 11.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435971 sha256=6339d9f34299ca90294e03efcb03e1f1c7d48dadfb577cf01df723241f27b529
  Stored in directory: /root/.cache/pip/wheels/7e/6d/0a/6b0

In [ ]:
import modin.pandas as pd
import collections, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan
# $example off$
from pyspark.sql import SparkSession,Row
from pyspark.sql.types import *
from pyspark.sql.functions import collect_set, col, count,explode,collect_list
#import pyspark.sql.functions as F

In [ ]:
spark = SparkSession\
.builder\
.appName("FPGrowthExample")\
.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
allergy = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/allergies.csv"
careplans = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/careplans.csv"
claims = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims.csv"
devices = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/devices.csv"
encounters = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/encounters.csv"
imaging_studies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/imaging_studies.csv"
immunuzation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/immunizations.csv"
medication = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/medications.csv"
observation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/observations.csv"
organization = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/organizations.csv"
patinet = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv"
payer_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payer_transitions.csv"
payers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payers.csv"
procedure = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/procedures.csv"
providers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/providers.csv"
supplies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/supplies.csv"
claim_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims_transactions.csv"
condition = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/conditions.csv"

In [ ]:
allergy = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(allergy))
careplans = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(careplans))
claims = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(claims))
devices = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(devices))
encounters = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(encounters))
imaging_studies = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(imaging_studies))
immunuzation = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(immunuzation))
medication = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(medication))
observation = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(observation))
organization = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(organization))
patient = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(patinet))
payer_transaction = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(payer_transaction))
payers = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(payers))
procedure = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(procedure))
providers = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(providers))
supplies = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(supplies))
claim_transaction = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(claim_transaction))
condition = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(condition))

In [ ]:
allergy.createOrReplaceTempView("aller")
careplans.createOrReplaceTempView("carepla")
claims.createOrReplaceTempView("clai")
devices.createOrReplaceTempView("devic")
encounters.createOrReplaceTempView("encounte")
imaging_studies.createOrReplaceTempView("imging_studi")
immunuzation.createOrReplaceTempView("immunizati")
medication.createOrReplaceTempView("medicati")
observation.createOrReplaceTempView("observati")
organization.createOrReplaceTempView("organizati")
patient.createOrReplaceTempView("patie")
payer_transaction.createOrReplaceTempView("payer_transacti")
payers.createOrReplaceTempView("paye")
procedure.createOrReplaceTempView("procedu")
providers.createOrReplaceTempView("provide")
supplies.createOrReplaceTempView("suppli")
claim_transaction.createOrReplaceTempView("claim_transacti")
condition.createOrReplaceTempView("conditi")

In [ ]:
pat_enco = spark.sql("select p.Id,p.FIRST,p.LAST,e.ENCOUNTERCLASS,e.TOTAL_CLAIM_COST from encounte e, patie p where p.Id == e.PATIENT")
#print(pat_data)
print(pat_enco.schema.names)
print(pat_enco.show())

['Id', 'FIRST', 'LAST', 'ENCOUNTERCLASS', 'TOTAL_CLAIM_COST']
+--------------------+----------+----------+--------------+----------------+
|                  Id|     FIRST|      LAST|ENCOUNTERCLASS|TOTAL_CLAIM_COST|
+--------------------+----------+----------+--------------+----------------+
|9b5de0da-9c38-358...|Gwyneth692|   Koch169|      wellness|         1481.66|
|231a2cb1-0ebe-dc1...|   Bill567|   Wolf938|      wellness|         1429.29|
|34c9600a-6f70-b55...| Víctor493|Rentería94|     emergency|         9092.99|
|231a2cb1-0ebe-dc1...|   Bill567|   Wolf938|      wellness|          786.33|
|9b5de0da-9c38-358...|Gwyneth692|   Koch169|      wellness|          786.33|
|9b5de0da-9c38-358...|Gwyneth692|   Koch169|      wellness|         1611.97|
|9b5de0da-9c38-358...|Gwyneth692|   Koch169|      wellness|         1335.14|
|9b5de0da-9c38-358...|Gwyneth692|   Koch169|      wellness|          786.33|
|34c9600a-6f70-b55...| Víctor493|Rentería94|      wellness|          878.67|
|9b5de0da-9c38

In [ ]:
pat_enco_class = pat_enco.groupBy('Id').agg(collect_list("TOTAL_CLAIM_COST").alias('TOTAL_CLAIM_COST'),collect_list("ENCOUNTERCLASS").alias('ENCOUNTERCLASS'))
pat_enco_class.createOrReplaceTempView('class1')
print(pat_enco_class.count())
pat_enco_class.show()

6038
+--------------------+--------------------+--------------------+
|                  Id|    TOTAL_CLAIM_COST|      ENCOUNTERCLASS|
+--------------------+--------------------+--------------------+
|050a7653-46cc-4b7...|[1408.55, 786.33,...|[wellness, wellne...|
|0b137dae-22a8-8f3...|[269.68, 791.04, ...|[wellness, wellne...|
|0e3c545f-2be9-63f...|[1493.54, 77.49, ...|[wellness, ambula...|
|17fbaf1c-1607-8c6...|[1511.12, 1462.86...|[wellness, wellne...|
|1f245d1f-3474-10b...|[129.16, 9001.82,...|[emergency, inpat...|
|231d9d40-9bff-9f1...|[77.49, 1207.19, ...|[ambulatory, well...|
|49374c03-3a21-8dd...|[77.49, 786.33, 1...|[ambulatory, well...|
|5792f730-1cb7-004...|[269.68, 1542.99,...|[wellness, wellne...|
|7521adae-a41a-b57...|[77.49, 48683.23,...|[ambulatory, emer...|
|7fdb3007-5d44-0c1...|[77.49, 77.49, 78...|[ambulatory, ambu...|
|93538c7f-287e-1ef...|[77.49, 786.33, 1...|[ambulatory, well...|
|9898cc43-82e7-adb...|[16461.74, 1162.4...|[emergency, emerg...|
|9fa6922e-fa65-32e..

In [ ]:
pat_encoDF = pat_enco.select("*").toPandas()
print(pat_encoDF['TOTAL_CLAIM_COST'].isna().sum() )
class1DF = pat_enco_class.select("*").toPandas()
print(pat_encoDF['TOTAL_CLAIM_COST'].isna().sum() )

0
0


In [ ]:
pat1_class2 = pat_enco.groupBy('Id').agg(collect_set('ENCOUNTERCLASS').alias('items'))
pat1_class2.createOrReplaceTempView('class2')
print(pat1_class2.count())
pat1_class2.show()

6038
+--------------------+--------------------+
|                  Id|               items|
+--------------------+--------------------+
|050a7653-46cc-4b7...|[outpatient, well...|
|0b137dae-22a8-8f3...|[outpatient, well...|
|0e3c545f-2be9-63f...|[outpatient, well...|
|17fbaf1c-1607-8c6...|[outpatient, well...|
|1f245d1f-3474-10b...|[outpatient, well...|
|231d9d40-9bff-9f1...|[outpatient, well...|
|49374c03-3a21-8dd...|[outpatient, well...|
|5792f730-1cb7-004...|[outpatient, well...|
|7521adae-a41a-b57...|[outpatient, well...|
|7fdb3007-5d44-0c1...|[outpatient, well...|
|93538c7f-287e-1ef...|[outpatient, well...|
|9898cc43-82e7-adb...|[outpatient, well...|
|9fa6922e-fa65-32e...|[wellness, ambula...|
|a23e7de2-1571-3bc...|[outpatient, well...|
|bff182c2-96a7-839...|[outpatient, well...|
|c5010808-c106-f4f...|[outpatient, well...|
|d3cb0101-bc8e-44f...|[outpatient, well...|
|1a576679-5063-ecb...|[outpatient, well...|
|23fb32d0-64dc-6c0...|[outpatient, well...|
|366ad69e-d8df-b2e...|[outp

In [ ]:
pat_Tcost = pat_enco.groupBy('Id').agg(collect_set('TOTAL_CLAIM_COST').alias('items'))
pat_Tcost.createOrReplaceTempView('class3')
print(pat_Tcost.count())
pat_Tcost.show()

6038
+--------------------+--------------------+
|                  Id|               items|
+--------------------+--------------------+
|050a7653-46cc-4b7...|[12521.3, 1521.0,...|
|0b137dae-22a8-8f3...|[1565.25, 815.87,...|
|0e3c545f-2be9-63f...|[0.0, 26896.11, 1...|
|17fbaf1c-1607-8c6...|[1824.32, 77.49, ...|
|1f245d1f-3474-10b...|[11111.05, 9720.7...|
|231d9d40-9bff-9f1...|[8929.53, 26928.2...|
|49374c03-3a21-8dd...|[1091.38, 1507.54...|
|5792f730-1cb7-004...|[13190.2, 16307.2...|
|7521adae-a41a-b57...|[1207.89, 2087.85...|
|7fdb3007-5d44-0c1...|[1296.24, 2435.6,...|
|93538c7f-287e-1ef...|[2832.36, 4186.14...|
|9898cc43-82e7-adb...|[0.0, 20673.36, 1...|
|9fa6922e-fa65-32e...|[1399.05, 714.05,...|
|a23e7de2-1571-3bc...|[645.81, 18881.58...|
|bff182c2-96a7-839...|[0.0, 17201.61, 1...|
|c5010808-c106-f4f...|[1978.9, 66646.19...|
|d3cb0101-bc8e-44f...|[1385.55, 129.16,...|
|1a576679-5063-ecb...|[1809.58, 1678.17...|
|23fb32d0-64dc-6c0...|[13864.83, 11204....|
|366ad69e-d8df-b2e...|[1162

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select c.items from class2 c")
################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_Encounter = model.associationRules
#rules_TClaim_cost.show()  
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF)#.show()
rules_Encounter.createOrReplaceTempView("rules_TClaim_cost_table")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
+--------------------+----+
|               items|freq|
+--------------------+----+
|          [wellness]|6031|
|        [ambulatory]|5949|
|[ambulatory, well...|5942|
|        [outpatient]|5111|
|[outpatient, ambu...|5089|
|[outpatient, ambu...|5082|
|[outpatient, well...|5104|
|         [emergency]|4370|
|[emergency, outpa...|3829|
|[emergency, outpa...|3822|
|[emergency, outpa...|3816|
|[emergency, outpa...|3823|
|[emergency, ambul...|4351|
|[emergency, ambul...|4345|
|[emergency, welln...|4364|
|         [inpatient]|2138|
|[inpatient, outpa...|1933|
|[inpatient, outpa...|1933|
|[inpatient, outpa...|1931|
|[inpatient, outpa...|1931|
+--------------------+----+
only showing top 20 rows



In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

####################### IT IS SHOWING PREDICTED DATA OF ENCOUNTERCLASS

# Extract out the items 
class1_itemsDF = spark.sql("select items from class3")#.as[Array[String]].toDF("items")

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_TClaim = model.associationRules
#rules_data.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF)
rules_TClaim.createOrReplaceTempView("rule_table")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
+--------------------+----+
|               items|freq|
+--------------------+----+
|             [77.49]|5815|
|            [786.33]|5106|
|     [786.33, 77.49]|5006|
|            [129.16]|4154|
|    [129.16, 786.33]|3472|
|[129.16, 786.33, ...|3415|
|     [129.16, 77.49]|4039|
|            [926.85]|3844|
|    [926.85, 129.16]|2580|
|[926.85, 129.16, ...|2491|
|[926.85, 129.16, ...|2453|
|[926.85, 129.16, ...|2539|
|    [926.85, 786.33]|3701|
|[926.85, 786.33, ...|3633|
|     [926.85, 77.49]|3771|
|            [269.68]|3018|
|    [269.68, 129.16]|2300|
|[269.68, 129.16, ...|1683|
|[269.68, 129.16, ...|1663|
|[269.68, 129.16, ...|2232|
+--------------------+----+
only showing top 20 rows



In [ ]:
rulesDF1 = rules_Encounter.select("*").toPandas()
rulesDF1.head()
rulesDF1.shape

(186, 4)

In [ ]:
rulesDF1[rulesDF1["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')

,antecedent,consequent,confidence,lift
85,"[urgentcare, emergency, outpatient, ambulatory...",[inpatient],0.485385,1.370792
12,"[urgentcare, emergency, outpatient, wellness]",[inpatient],0.485385,1.370792
150,"[urgentcare, emergency, outpatient]",[inpatient],0.485385,1.370792
16,"[urgentcare, emergency, outpatient, ambulatory]",[inpatient],0.485385,1.370792
68,"[urgentcare, emergency, ambulatory]",[inpatient],0.475983,1.344239
...,...,...,...,...
58,"[urgentcare, ambulatory]",[wellness],1.000000,1.001161
124,"[urgentcare, inpatient, emergency, outpatient,...",[wellness],1.000000,1.001161
122,"[urgentcare, inpatient, outpatient]",[wellness],1.000000,1.001161
70,"[urgentcare, emergency, ambulatory]",[wellness],1.000000,1.001161


In [ ]:
rulesTC_cost = rules_TClaim.select("*").toPandas()
rulesTC_cost.head()
rulesTC_cost.shape

(15399, 4)

In [ ]:
rulesTC_cost[rulesTC_cost["antecedent"].apply(lambda x: "926.85" in str(x))].sort_values(ascending=False,by='lift')

,antecedent,consequent,confidence,lift
1757,"[831.76, 269.68, 926.85, 77.49]",[691.24],0.533333,7.046535
892,"[831.76, 269.68, 926.85]",[691.24],0.532468,7.035096
14866,"[831.76, 926.85, 77.49]",[691.24],0.526316,6.953818
11560,"[831.76, 926.85]",[691.24],0.525641,6.944903
15365,"[594.14, 269.68, 926.85, 129.16, 786.33, 77.49]",[550.72],0.069643,3.391158
...,...,...,...,...
5046,"[218.01, 926.85, 786.33, 77.49]",[691.24],0.025739,0.340074
8617,"[926.85, 129.16]",[831.76],0.021705,0.185634
12125,"[926.85, 129.16, 77.49]",[831.76],0.021268,0.181894
9617,[926.85],[831.76],0.020291,0.173540


In [ ]:
dataframe = pd.concat([rulesTC_cost, rulesDF1], axis=0)

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



In [ ]:
dataframe.iloc[15398,:]

antecedent    [1067.37, 1162.46, 269.68, 129.16, 786.33, 77.49]
consequent                                             [645.81]
confidence                                             0.727273
lift                                                   1.728167
Name: 15398, dtype: object

In [ ]:
pat_alg = spark.sql("select p.Id, alg.DESCRIPTION from aller alg, patie p where p.Id == alg.PATIENT")
pat_alg.show()

+--------------------+--------------------+
|                  Id|         DESCRIPTION|
+--------------------+--------------------+
|1e2b4480-645f-f03...|Bee venom (substa...|
|1e2b4480-645f-f03...|     Mold (organism)|
|1e2b4480-645f-f03...|House dust mite (...|
|1e2b4480-645f-f03...|Animal dander (su...|
|1e2b4480-645f-f03...|Grass pollen (sub...|
|1e2b4480-645f-f03...|Tree pollen (subs...|
|1e2b4480-645f-f03...|             Aspirin|
|1e2b4480-645f-f03...|Eggs (edible) (su...|
|1e2b4480-645f-f03...|   Wheat (substance)|
|1e2b4480-645f-f03...|    Fish (substance)|
|63a4ee1d-0a99-792...|Shellfish (substa...|
|cf323fd4-a173-ae6...|             Aspirin|
|e363961e-e69e-66d...|   Latex (substance)|
|e363961e-e69e-66d...|     Mold (organism)|
|e363961e-e69e-66d...|Animal dander (su...|
|e363961e-e69e-66d...|Grass pollen (sub...|
|e363961e-e69e-66d...|Tree pollen (subs...|
|e363961e-e69e-66d...|            cefdinir|
|e363961e-e69e-66d...|        Penicillin V|
|e363961e-e69e-66d...|   Wheat (

In [ ]:
pat_alg_des = pat_alg.groupBy('Id').agg(collect_set('DESCRIPTION').alias('items'))
pat_alg_des.createOrReplaceTempView('alg_des')
print(pat_alg_des.count())
pat_alg_des.show()

865
+--------------------+--------------------+
|                  Id|               items|
+--------------------+--------------------+
|0b137dae-22a8-8f3...|[Aspirin, House d...|
|69aad4d9-6ddb-e13...|[Aspirin, Shellfi...|
|7fdb3007-5d44-0c1...|[Soya bean (subst...|
|8b8817c3-ccf8-0e5...|[Tree pollen (sub...|
|c5010808-c106-f4f...|[Latex (substance...|
|d7ae8592-4883-1fa...|[House dust mite ...|
|dc5bca7e-b102-9a5...|[Shellfish (subst...|
|296db11a-2bf8-7c9...|[Aspirin, Cow's m...|
|32ae76a7-97d8-5b2...|[Shellfish (subst...|
|3a5c110b-440c-50a...|[Shellfish (subst...|
|428648f7-df01-58c...|[Aspirin, Grass p...|
|570ef78b-27e7-9f7...|          [cefdinir]|
|6f20f149-b9e0-2b6...|[Shellfish (subst...|
|8ea91684-6822-48e...|[Shellfish (subst...|
|b869f8ac-8310-135...| [Wheat (substance)]|
|9cd166ef-b524-10d...|[House dust mite ...|
|c95908c9-fb6c-0b0...|[Aspirin, Bee ven...|
|de87ee28-69a7-b66...|[Aspirin, Grass p...|
|e068ce30-2961-2c5...|[Peanut (substanc...|
|5f5c0bc7-d468-db5...|[Grass

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from alg_des")#.as[Array[String]].toDF("items")

################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_alg_des = model.associationRules
#rules_TClaim_cost.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_alg_des.createOrReplaceTempView("rules_TClaim_cost_table")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
+--------------------+----+
|               items|freq|
+--------------------+----+
|   [Mold (organism)]| 523|
|[Animal dander (s...| 501|
|[Animal dander (s...| 454|
|[Grass pollen (su...| 411|
|[Grass pollen (su...| 393|
|[Grass pollen (su...| 391|
|[Grass pollen (su...| 392|
|[House dust mite ...| 397|
|[House dust mite ...| 341|
|[House dust mite ...| 341|
|[House dust mite ...| 341|
|[House dust mite ...| 341|
|[House dust mite ...| 389|
|[House dust mite ...| 389|
|[House dust mite ...| 390|
|[Tree pollen (sub...| 388|
|[Tree pollen (sub...| 330|
|[Tree pollen (sub...| 330|
|[Tree pollen (sub...| 330|
|[Tree pollen (sub...| 330|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+
|               items|          prediction|
+--------------------+--------------------+
|[Aspirin, House d...|[Latex (substance...|
|[Aspirin, Shellfi...|[Latex (substance...|
|[Soya bean (subst

In [ ]:
rules_alg_DF = rules_alg_des.select("*").toPandas()
rules_alg_DF.head()
rules_alg_DF.shape

(29575, 4)

In [ ]:
rules_alg_DF.iat[1,1]

['Bee venom (substance)']

In [ ]:
rules_alg_DF[rules_alg_DF["antecedent"].apply(lambda x: "Bee venom (substance)" in str(x))].sort_values(ascending=False,by='lift')

,antecedent,consequent,confidence,lift
25021,"[Eggs (edible) (substance), Bee venom (substan...",[Fish (substance)],0.857143,5.372671
12027,"[Eggs (edible) (substance), Bee venom (substan...",[Fish (substance)],0.857143,5.372671
14883,"[Eggs (edible) (substance), Bee venom (substan...",[Fish (substance)],0.857143,5.372671
23574,"[Eggs (edible) (substance), Bee venom (substan...",[Fish (substance)],0.857143,5.372671
29018,"[Eggs (edible) (substance), Bee venom (substan...",[Fish (substance)],0.857143,5.372671
...,...,...,...,...
21816,"[Bee venom (substance), Aspirin]",[Shellfish (substance)],0.150000,0.566594
27388,"[Eggs (edible) (substance), Bee venom (substan...",[Shellfish (substance)],0.136364,0.515085
7822,"[Eggs (edible) (substance), Bee venom (substan...",[Shellfish (substance)],0.136364,0.515085
28946,"[Eggs (edible) (substance), Bee venom (substan...",[Shellfish (substance)],0.136364,0.515085


In [ ]:
#pat_car = spark.sql("select p.Id from carepla car, patie p where p.Id == car.PATIENTID")
#pat_car.show()

In [ ]:
pat_clmtr = spark.sql("select p.Id, clmtr.TYPE, clmtr.NOTES from claim_transacti clmtr, patie p where p.Id == clmtr.PATIENTID")
pat_clmtr.show()

+--------------------+-----------+--------------------+
|                  Id|       TYPE|               NOTES|
+--------------------+-----------+--------------------+
|231a2cb1-0ebe-dc1...|     CHARGE|General examinati...|
|231a2cb1-0ebe-dc1...|TRANSFEROUT|General examinati...|
|231a2cb1-0ebe-dc1...| TRANSFERIN|General examinati...|
|9b5de0da-9c38-358...|     CHARGE|General examinati...|
|34c9600a-6f70-b55...|     CHARGE|Emergency Room Ad...|
|9b5de0da-9c38-358...|    PAYMENT|General examinati...|
|231a2cb1-0ebe-dc1...|    PAYMENT|General examinati...|
|9b5de0da-9c38-358...|TRANSFEROUT|General examinati...|
|9b5de0da-9c38-358...| TRANSFERIN|General examinati...|
|9b5de0da-9c38-358...|    PAYMENT|General examinati...|
|231a2cb1-0ebe-dc1...|     CHARGE|General examinati...|
|231a2cb1-0ebe-dc1...|TRANSFEROUT|General examinati...|
|231a2cb1-0ebe-dc1...| TRANSFERIN|General examinati...|
|231a2cb1-0ebe-dc1...|    PAYMENT|General examinati...|
|231a2cb1-0ebe-dc1...|     CHARGE|Influenza  sea

In [ ]:
pat_clmtr1 = pat_clmtr.groupBy('Id').agg(collect_set('TYPE').alias('items'))
pat_clmtr1.show()
pat_clmtr1.createOrReplaceTempView('clmtr_type')
print(pat_clmtr1.count())
pat_clmtr1.show()

+--------------------+--------------------+
|                  Id|               items|
+--------------------+--------------------+
|7fdb3007-5d44-0c1...|[TRANSFEROUT, CHA...|
|a23e7de2-1571-3bc...|[TRANSFEROUT, CHA...|
|9898cc43-82e7-adb...|[TRANSFEROUT, CHA...|
|bff182c2-96a7-839...|[TRANSFEROUT, CHA...|
|17fbaf1c-1607-8c6...|[TRANSFEROUT, CHA...|
|231d9d40-9bff-9f1...|[TRANSFEROUT, CHA...|
|7521adae-a41a-b57...|[TRANSFEROUT, CHA...|
|0b137dae-22a8-8f3...|[TRANSFEROUT, CHA...|
|49374c03-3a21-8dd...|[TRANSFEROUT, CHA...|
|5792f730-1cb7-004...|[TRANSFEROUT, CHA...|
|9fa6922e-fa65-32e...|[TRANSFEROUT, CHA...|
|050a7653-46cc-4b7...|[TRANSFEROUT, CHA...|
|0e3c545f-2be9-63f...|[TRANSFEROUT, CHA...|
|d3cb0101-bc8e-44f...|[TRANSFEROUT, CHA...|
|1f245d1f-3474-10b...|[TRANSFEROUT, CHA...|
|93538c7f-287e-1ef...|[TRANSFEROUT, CHA...|
|c5010808-c106-f4f...|[TRANSFEROUT, CHA...|
|69aad4d9-6ddb-e13...|[TRANSFEROUT, CHA...|
|dc5bca7e-b102-9a5...|[TRANSFEROUT, CHA...|
|366ad69e-d8df-b2e...|[TRANSFERO

In [ ]:
pat_clmtr2 = pat_clmtr.groupBy('Id').agg(collect_set('NOTES').alias('items'))
pat_clmtr2.createOrReplaceTempView('clmtr_note')
print(pat_clmtr2.count())
pat_clmtr2.show()

6038
+--------------------+--------------------+
|                  Id|               items|
+--------------------+--------------------+
|7fdb3007-5d44-0c1...|[Encounter for ch...|
|a23e7de2-1571-3bc...|[Bone immobilizat...|
|9898cc43-82e7-adb...|[Prenatal visit, ...|
|bff182c2-96a7-839...|[Prenatal visit, ...|
|17fbaf1c-1607-8c6...|[Depression scree...|
|231d9d40-9bff-9f1...|[Encounter for ch...|
|7521adae-a41a-b57...|[Encounter for ch...|
|0b137dae-22a8-8f3...|[Loratadine 5 MG ...|
|49374c03-3a21-8dd...|[Encounter for ch...|
|5792f730-1cb7-004...|[Encounter for ch...|
|9fa6922e-fa65-32e...|[IPV, Influenza  ...|
|050a7653-46cc-4b7...|[Encounter for ch...|
|0e3c545f-2be9-63f...|[Screening for oc...|
|d3cb0101-bc8e-44f...|[Depression scree...|
|1f245d1f-3474-10b...|[Encounter for ch...|
|93538c7f-287e-1ef...|[Prenatal visit, ...|
|c5010808-c106-f4f...|[Encounter for ch...|
|69aad4d9-6ddb-e13...|[Encounter for ch...|
|dc5bca7e-b102-9a5...|[Depression scree...|
|366ad69e-d8df-b2e...|[Enco

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from clmtr_type")
################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_clmtr_type = model.associationRules
#rules_TClaim_cost.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_clmtr_type.createOrReplaceTempView("rules_clmtr_type")

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs
+--------------------+----+
|               items|freq|
+--------------------+----+
|            [CHARGE]|6038|
|           [PAYMENT]|6038|
|   [PAYMENT, CHARGE]|6038|
|       [TRANSFEROUT]|5960|
|[TRANSFEROUT, PAY...|5960|
|[TRANSFEROUT, PAY...|5960|
|[TRANSFEROUT, CHA...|5960|
|        [TRANSFERIN]|5960|
|[TRANSFERIN, TRAN...|5960|
|[TRANSFERIN, TRAN...|5960|
|[TRANSFERIN, TRAN...|5960|
|[TRANSFERIN, TRAN...|5960|
|[TRANSFERIN, PAYM...|5960|
|[TRANSFERIN, PAYM...|5960|
|[TRANSFERIN, CHARGE]|5960|
+--------------------+----+

+--------------------+----------+
|               items|prediction|
+--------------------+----------+
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|        []|
|[TRANSFEROUT, CHA...|       

In [ ]:
#%scala
%time
#from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from clmtr_note")

################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_clmtr_note = model.associationRules
#rules_TClaim_cost.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_clmtr_note.createOrReplaceTempView("rules_clmtr_note")

In [ ]:
rules_clmtr_type = rules_clmtr_type.select("*").toPandas()
print(rules_clmtr_type.shape)
rules_clmtr_type.head()

In [ ]:
rules_clmtr_type.iat[1,1]

In [ ]:
rules_clmtr_type[rules_clmtr_type["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
rulesdf_clmtr_note = rules_clmtr_note.select("*").toPandas()
rulesdf_clmtr_note.head()
rulesdf_clmtr_note.shape

In [ ]:
rulesdf_clmtr_note.iat[1,1]

In [ ]:
rulesdf_clmtr_note[rulesdf_clmtr_note["antecedent"].apply(lambda x: "926.85" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
dataframe = pd.concat([rules_clmtr_type, rulesdf_clmtr_note], axis=0)

In [ ]:
dataframe.head()

In [ ]:
pat_cond = spark.sql("select p.Id, cond.DESCRIPTION from conditi cond, patie p where p.Id == cond.PATIENT")
pat_cond.show()

+--------------------+--------------------+
|                  Id|         DESCRIPTION|
+--------------------+--------------------+
|231a2cb1-0ebe-dc1...|Received higher e...|
|231a2cb1-0ebe-dc1...|Full-time employm...|
|231a2cb1-0ebe-dc1...|Lack of access to...|
|231a2cb1-0ebe-dc1...|Limited social co...|
|231a2cb1-0ebe-dc1...|    Stress (finding)|
|9b5de0da-9c38-358...|Received higher e...|
|9b5de0da-9c38-358...|Transport problem...|
|9b5de0da-9c38-358...|Full-time employm...|
|9b5de0da-9c38-358...|Lack of access to...|
|231a2cb1-0ebe-dc1...|Part-time employm...|
|231a2cb1-0ebe-dc1...|Transport problem...|
|9b5de0da-9c38-358...|Limited social co...|
|9b5de0da-9c38-358...|    Stress (finding)|
|9b5de0da-9c38-358...|Has a criminal re...|
|34c9600a-6f70-b55...|    Seizure disorder|
|34c9600a-6f70-b55...|History of single...|
|9b5de0da-9c38-358...|Full-time employm...|
|9b5de0da-9c38-358...|Social isolation ...|
|9b5de0da-9c38-358...|Streptococcal sor...|
|9b5de0da-9c38-358...|Body mass 

In [ ]:
pat_cond1 = pat_cond.groupBy('Id').agg(collect_set('DESCRIPTION').alias('items'))
pat_cond1.createOrReplaceTempView('pat_cond_des')
print(pat_cond1.count())
pat_cond1.show()

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from pat_cond_des")#.as[Array[String]].toDF("items")

################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_cond = model.associationRules
#rules_TClaim_cost.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_cond.createOrReplaceTempView("rules_cond")

In [ ]:
rulesDF_cond = rules_data.select("*").toPandas()
print(rulesDF_cond.shape)
rulesDF_cond.head()

In [ ]:
rulesDF_cond.iat[1,1]

In [ ]:
rulesDF_cond[rulesDF_cond["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
pat_cond = spark.sql("select p.Id, cond.DESCRIPTION from conditi cond, patie p where p.Id == cond.PATIENT")
pat_cond.show()

+--------------------+--------------------+
|                  Id|         DESCRIPTION|
+--------------------+--------------------+
|231a2cb1-0ebe-dc1...|Received higher e...|
|231a2cb1-0ebe-dc1...|Full-time employm...|
|231a2cb1-0ebe-dc1...|Lack of access to...|
|231a2cb1-0ebe-dc1...|Limited social co...|
|231a2cb1-0ebe-dc1...|    Stress (finding)|
|9b5de0da-9c38-358...|Received higher e...|
|9b5de0da-9c38-358...|Transport problem...|
|9b5de0da-9c38-358...|Full-time employm...|
|9b5de0da-9c38-358...|Lack of access to...|
|231a2cb1-0ebe-dc1...|Part-time employm...|
|231a2cb1-0ebe-dc1...|Transport problem...|
|9b5de0da-9c38-358...|Limited social co...|
|9b5de0da-9c38-358...|    Stress (finding)|
|9b5de0da-9c38-358...|Has a criminal re...|
|34c9600a-6f70-b55...|    Seizure disorder|
|34c9600a-6f70-b55...|History of single...|
|9b5de0da-9c38-358...|Full-time employm...|
|9b5de0da-9c38-358...|Social isolation ...|
|9b5de0da-9c38-358...|Streptococcal sor...|
|9b5de0da-9c38-358...|Body mass 

In [ ]:
#pat_dev = spark.sql("select p.Id, dev.DESCRIPTION from devic dev, patie p where p.Id == dev.PATIENT")
#pat_dev.show()

In [ ]:
pat_imgstd = spark.sql("select p.Id, imgstd.BODYSITE_DESCRIPTION from imging_studi imgstd, patie p where p.Id == imgstd.PATIENT")
pat_imgstd.show()

+--------------------+--------------------+
|                  Id|BODYSITE_DESCRIPTION|
+--------------------+--------------------+
|231a2cb1-0ebe-dc1...|               Chest|
|a48808e4-6d82-a34...|               Chest|
|f7be139e-db02-e25...|               Wrist|
|c2d5edb6-c0fb-584...|                 Arm|
|c50b03b3-5933-89a...|              Pelvis|
|fab37d10-f152-309...|                 Arm|
|235eeab6-822b-ca6...|Heart structure (...|
|235eeab6-822b-ca6...|Heart structure (...|
|235eeab6-822b-ca6...|Thoracic structur...|
|235eeab6-822b-ca6...|Heart structure (...|
|235eeab6-822b-ca6...|Thoracic structur...|
|235eeab6-822b-ca6...|Heart structure (...|
|235eeab6-822b-ca6...|Heart structure (...|
|235eeab6-822b-ca6...|                Knee|
|235eeab6-822b-ca6...|Thoracic structur...|
|235eeab6-822b-ca6...|Heart structure (...|
|a66cc082-0eb8-09d...|               Chest|
|e363961e-e69e-66d...|                 Arm|
|99397bcd-e5b3-ac8...|                 Arm|
|9884917a-373d-9c7...|          

In [ ]:
#immunizations
#pat_imgstd = spark.sql("select p.Id, imgstd.BODYSITE_DESCRIPTION from imging_studi imgstd, patie p where p.Id == imgstd.PATIENT")
#pat_imgstd.show()

In [ ]:
#medications
#pat_imgstd = spark.sql("select p.Id, imgstd.BODYSITE_DESCRIPTION from imging_studi imgstd, patie p where p.Id == imgstd.PATIENT")
#pat_imgstd.show()

In [ ]:
#observations
pat_obs = spark.sql("select p.Id, obs.DESCRIPTION from observati obs, patie p where p.Id == obs.PATIENT")
pat_obs.show()

+--------------------+--------------------+
|                  Id|         DESCRIPTION|
+--------------------+--------------------+
|231a2cb1-0ebe-dc1...|         Body Height|
|231a2cb1-0ebe-dc1...|Pain severity - 0...|
|231a2cb1-0ebe-dc1...|         Body Weight|
|231a2cb1-0ebe-dc1...|     Body Mass Index|
|231a2cb1-0ebe-dc1...|Body mass index (...|
|231a2cb1-0ebe-dc1...|Diastolic Blood P...|
|231a2cb1-0ebe-dc1...|Systolic Blood Pr...|
|231a2cb1-0ebe-dc1...|          Heart rate|
|231a2cb1-0ebe-dc1...|    Respiratory rate|
|231a2cb1-0ebe-dc1...|Tobacco smoking s...|
|231a2cb1-0ebe-dc1...|In the past year ...|
|231a2cb1-0ebe-dc1...|Do you feel physi...|
|231a2cb1-0ebe-dc1...|  Are you a refugee?|
|231a2cb1-0ebe-dc1...|In the past year ...|
|231a2cb1-0ebe-dc1...|Stress is when so...|
|231a2cb1-0ebe-dc1...|How often do you ...|
|231a2cb1-0ebe-dc1...|Has lack of trans...|
|231a2cb1-0ebe-dc1...|In the past year ...|
|231a2cb1-0ebe-dc1...|During the past y...|
|231a2cb1-0ebe-dc1...|What is yo

In [ ]:
#organization
#it contain no Patient Foreign key

In [ ]:
#payer_transitions
#it contain no related data for Common Disease Pattern

In [ ]:
#payers
#it contian not related data for Common Disease Pattern 

In [ ]:
#procedures
pat_proc = spark.sql("select p.Id, proc.DESCRIPTION from procedu proc, patie p where p.Id == proc.PATIENT")
pat_proc.show()

+--------------------+--------------------+
|                  Id|         DESCRIPTION|
+--------------------+--------------------+
|231a2cb1-0ebe-dc1...|Assessment of hea...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Assessment of sub...|
|231a2cb1-0ebe-dc1...|Assessment using ...|
|231a2cb1-0ebe-dc1...|Assessment of hea...|
|231a2cb1-0ebe-dc1...|Medication Reconc...|
|231a2cb1-0ebe-dc1...|Assessment of anx...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Assessment of sub...|
|231a2cb1-0ebe-dc1...|Assessment using ...|
|34c9600a-6f70-b55...|Medication Reconc...|
|231a2cb1-0ebe-dc1...|         Chest X-ray|
|231a2cb1-0ebe-dc1...| Bone immobilization|
|231a2cb1-0ebe-dc1...|Assessment of hea...|
|231a2cb1-0ebe-dc1...|Medication Reconc...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Depression screen...|
|231a2cb1-0ebe-dc1...|Assessment

In [ ]:
#providers
#it does not contain patient foreign key

In [ ]:
#supplies
#it contains the description of supplies for treatment 

In [ ]:
pat_proc.count()

472490